In [1]:
import tensorflow as tf
tf.__version__

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.0'

In [2]:
from tensorflow.contrib import keras

In [3]:
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
import time
import os

np.random.seed(1337)

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from numpy import nan
import sys
import datetime

now = datetime.datetime.now

In [5]:
from matplotlib import pyplot as plt

%matplotlib inline

In [6]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [7]:
img_rows, img_cols = IMG_WIDTH, IMG_HEIGHT

if K.image_data_format() == 'channels_first':
    shape_ord = (3, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)


In [8]:
shape_ord

(224, 224, 3)

In [9]:
from keras.applications import vgg16
from keras.layers import Input

# build the VGG16 network with ImageNet weights
vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input((224, 224, 3)))
print('Model loaded.')

Model loaded.


In [10]:
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization

In [11]:
for l in vgg16.layers:
    l.trainable = False

In [15]:
x = Flatten(input_shape=vgg16.output.shape)(vgg16.output)
# x = Dense(4096, activation='relu', name='fully_connected_1')(x)
# x = Dropout(0.5)(x)
x = Dense(2048, activation='relu', name='fully_connected_1')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='fully_connected_2')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(4, activation = 'softmax')(x)

In [16]:
from keras.models import Model

model = Model(inputs=vgg16.input, outputs=predictions)

#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
train_datagen = ImageDataGenerator(
        zoom_range=0.2,
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        horizontal_flip=True,
        vertical_flip=True,
        )

In [16]:
train_generator = train_datagen.flow_from_directory(
        '../data/train',
        target_size=(224, 224),
        batch_size=32, #changed from 1, change back if needed
        class_mode='categorical')

Found 2491 images belonging to 4 classes.


In [17]:
validation_datagen = ImageDataGenerator()

In [18]:
validation_generator = validation_datagen.flow_from_directory(
        '../data/validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 846 images belonging to 4 classes.


In [19]:
test_datagen = ImageDataGenerator()

In [20]:
test_generator = test_datagen.flow_from_directory(
        '../data/test',
        target_size=(224,224),
        batch_size=32,
        class_mode='categorical')

Found 377 images belonging to 4 classes.


In [21]:
from keras.preprocessing import image

In [22]:
model.fit_generator(
        train_generator,
        steps_per_epoch=77,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=6,
        use_multiprocessing=True)

Epoch 1/20
 78/118 [==================>...........] - ETA: 38s - loss: 0.3542 - acc: 0.8630 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00

In [23]:
for l in model.layers:
    l.trainable = True
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit_generator(
        train_generator,
        steps_per_epoch=77,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=26,
        use_multiprocessing=True)

Epoch 1/20
78/77 [==============================] - 128s 2s/step - loss: 1.7467 - acc: 0.3263 - val_loss: 1.4874 - val_acc: 0.3085
Epoch 2/20
78/77 [==============================] - 116s 1s/step - loss: 1.5617 - acc: 0.3124 - val_loss: 3.1769 - val_acc: 0.2577
Epoch 3/20
78/77 [==============================] - 116s 1s/step - loss: 1.4386 - acc: 0.3514 - val_loss: 1.3189 - val_acc: 0.3251
Epoch 4/20
78/77 [==============================] - 116s 1s/step - loss: 1.4188 - acc: 0.3369 - val_loss: 1.4653 - val_acc: 0.3499
Epoch 5/20
78/77 [==============================] - 116s 1s/step - loss: 1.3982 - acc: 0.3560 - val_loss: 1.2900 - val_acc: 0.4338
Epoch 6/20
78/77 [==============================] - 110s 1s/step - loss: 1.4065 - acc: 0.3458 - val_loss: 1.4939 - val_acc: 0.3652
Epoch 7/20
78/77 [==============================] - 116s 1s/step - loss: 1.3841 - acc: 0.3516 - val_loss: 1.3298 - val_acc: 0.3333
Epoch 8/20
78/77 [==============================] - 116s 1s/step - loss: 1.3908 - a

In [24]:
# from sklearn.model_selection import GridSearchCV
# from keras.wrappers.scikit_learn import KerasClassifier

In [170]:
model_history = _

In [25]:
from sklearn.metrics import classification_report

In [26]:
model.evaluate_generator(validation_generator,steps=26)

[1.556619799447116, 0.29669030739905988]

In [173]:
import pandas as pd
df = pd.read_csv('../data/test/xy.txt')
df.head()

,file_name,order
0,photo_553054.jpg,Plecoptera
1,photo_585610.jpg,Diptera
2,photo_9719.jpg,Diptera
3,picture4281.jpg,Ephemeroptera
4,picture1347.jpg,Diptera


In [174]:
import cv2

In [175]:
t = test_generator.class_indices

In [176]:
y_cat = np_utils.to_categorical([t[i] for i in df.order], 4)
y_cat

array([[ 0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.]])

In [177]:
iml = []
for f,o in zip(df.file_name,df.order):
    i_path = '../data/test/{}/{}'.format(o,f)
    iml.append(cv2.resize(cv2.imread(i_path,1),(224,224),interpolation = cv2.INTER_AREA))

In [178]:
#iml = np.array(iml)
y_cat = np.array(y_cat)
y_true = []
y_pred = []

counter = 0
# for xbatch, ybatch in test_datagen.flow(iml,y_cat, batch_size = 32):
#     y_true.append(ybatch)
#     y_pred.append(model.predict(xbatch))
#     if counter == 16:
#         break
#     counter += 1

iml = np.stack(iml)

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(iml,y_cat,
        batch_size=32)


In [179]:
pred = model.predict(iml)

In [180]:
inv_map = {v: k for k, v in t.items()}
y_pred = [inv_map[np.argmax(i)] for i in pred]

In [181]:
y_true = [inv_map[np.argmax(i)] for i in y_cat]

In [182]:
'''
_____________

From RUN TWO
_____________
'''

print(classification_report(y_true,y_pred))

               precision    recall  f1-score   support

      Diptera       0.75      0.91      0.82       116
Ephemeroptera       0.99      0.67      0.80       104
   Plecoptera       0.82      0.79      0.81        82
  Trichoptera       0.70      0.79      0.75        78

  avg / total       0.82      0.80      0.80       380



In [147]:
'''
_____________

From RUN ONE
_____________
'''
print(classification_report(y_true,y_pred))

               precision    recall  f1-score   support

      Diptera       0.79      0.91      0.85       113
Ephemeroptera       0.93      0.86      0.89       214
   Plecoptera       0.73      0.92      0.81        83
  Trichoptera       0.92      0.73      0.82       100

  avg / total       0.87      0.85      0.85       510



In [6]:
from keras.applications import vgg16
from keras.layers import Input

from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization

# build the VGG16 network with ImageNet weights
vgg2 = vgg16.VGG16(include_top=False,weights='imagenet',classes=4,input_tensor=Input((224,224,3)))
print('Model loaded.')

Model loaded.


In [7]:
vgg2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [40]:
# vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input((224, 224, 3)))
# print('Model loaded.')

In [8]:
for l in vgg2.layers:
    l.trainable = False

In [9]:
x = Flatten(input_shape=vgg2.output.shape)(vgg2.output)
x = Dense(4096, activation='relu', name='ft_fc1')(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu', name='ft_fc2')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(4, activation = 'softmax')(x)

In [20]:
from keras.models import Model

model = Model(inputs=vgg2.input, outputs=predictions)

#compile the model
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
train_datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

In [13]:
train_generator = train_datagen.flow_from_directory(
        '../data/train2',
        target_size=(224, 224),
        batch_size=32, #changed from 1, change back if needed
        class_mode='categorical')

Found 1506 images belonging to 5 classes.


In [14]:
validation_datagen = ImageDataGenerator()

In [15]:
validation_generator = validation_datagen.flow_from_directory(
        '../data/validation2',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 502 images belonging to 5 classes.


In [16]:
test_datagen = ImageDataGenerator()

In [17]:
test_generator = test_datagen.flow_from_directory(
        '../data/test2',
        target_size=(224,224),
        batch_size=32,
        class_mode='categorical')

Found 224 images belonging to 5 classes.


In [18]:
model.fit_generator(
        train_generator,
        steps_per_epoch=118,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=6,
        use_multiprocessing=True)

Epoch 1/20


ValueError: Error when checking target: expected dense_1 to have shape (None, 4) but got array with shape (32, 5)

In [ ]:
for l in model.layers:
    l.trainable = True
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=118,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=6,
        use_multiprocessing=True)

In [25]:
# def make_model(dense_layer_sizes, dropout_sizes):
#     '''Creates model comprised of 2 convolutional layers followed by dense layers

#     dense_layer_sizes: List of layer sizes. This list has one number for each layer
#     nb_filters: Number of convolutional filters in each convolutional layer
#     nb_conv: Convolutional kernel size
#     nb_pool: Size of pooling area for max pooling
#     '''
    
#     x = Flatten(input_shape=vgg16.output.shape)(vgg16.output)
#     x = Dense(4096, activation='relu', name='ft_fc1')(x)
#     x = Dropout(0.5)(x)
#     x = Dense(2048, activation='relu', name='ft_fc12')(x)
#     x = Dropout(0.5)(x)
#     x = BatchNormalization()(x)
#     predictions = Dense(4, activation = 'softmax')(x)
    
#     for layer_size, d in dense_layer_sizes, dropout_sizes:
#         x = Dense(layer_size, activation = 'relu')(x)
#         x = Dropout(d)(x)
    
    
    
#     predictions = Dense(4, activation= 'softmax')(x)
#     model = Model(inputs=vgg16.input, outputs=predictions)

    
# #     model = Sequential()

# #     model.add(Conv2D(filters, (kernel_size, kernel_size),
# #                      padding='valid', input_shape=input_shape))
# #     model.add(Activation('relu'))
# #     model.add(Conv2D(filters, (kernel_size, kernel_size)))
# #     model.add(Activation('relu'))
# #     model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
# #     model.add(Dropout(0.25))

# #     model.add(Flatten())
# #     for layer_size in dense_layer_sizes:
# #         model.add(Dense(layer_size))
# #         model.add(Activation('relu'))
# #     model.add(Dropout(0.5))
# #     model.add(Dense(nb_classes))
# #     model.add(Activation('softmax'))

#     model.compile(loss='categorical_crossentropy',
#                   optimizer='adadelta',
#                   metrics=['accuracy'])

#     return model

In [26]:
dense_size_candidates = [[512,512],[1024,512],[1024,1024],[2048,1024]]
dropout_sizes = [[.5,.5],[.5,.25],[.25,.5],[.75,.5]]
my_classifier = KerasClassifier(make_model, batch_size=32)

In [28]:
validator = GridSearchCV(my_classifier,
                        param_grid={'dense_layer_sizes':dense_size_candidates,
                                    'dropout_sizes':dropout_sizes},
                         scoring='f1_samples',
                         n_jobs=-1
                        )

In [23]:
# validator.fit(
#         train_generator,
#         steps_per_epoch=118,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=10,
#         use_multiprocessing=True)